<a href="https://colab.research.google.com/github/PerimeterMaison/Assignment-4/blob/main/Assignment_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


OBS: CTRL+F ASDF. ALLT SOM HAR ASDF EFTER SIG MÅSTE KORRIGERAS!

/MoMoney

# DAT565 Introduction to Data Science and AI
## 2023-2024, LP2
## Assignment 4: Spam classification using Naïve Bayes
This assignment has three obligatory questions. Questions 4-5 are optional and will not be graded.


The exercise takes place in this notebook environment where you can chose to use Jupyter or Google Colabs. We recommend you use Google Colabs as it will facilitate remote group-work and makes the assignment less technical.

*Tips:*
* You can execute certain Linux shell commands by prefixing the command with a `!`.
* You can insert Markdown cells and code cells. The first you can use for documenting and explaining your results, the second you can use to write code snippets that execute the tasks required.  

In this assignment you will implement a Naïve Bayes classifier in Python that will classify emails into spam and non-spam (“ham”) classes.  Your program should be able to train on a given set of spam and “ham” datasets.

You will work with the datasets available at https://spamassassin.apache.org/old/publiccorpus/. There are three types of files in this location:
-	easy-ham: non-spam messages typically quite easy to differentiate from spam messages.
-	hard-ham: non-spam messages more difficult to differentiate
-	spam: spam messages

**Execute the cell below to download and extract the data into the environment of the notebook -- it will take a few seconds.**

If you chose to use Jupyter notebooks you will have to run the commands in the cell below on your local computer. Note that if you are using Windows, you can instead use [7zip](https://www.7-zip.org/download.html) to decompress the data (you will have to modify the cell below).

**What to submit:**
* Convert the notebook to a PDF file by compiling it, and submit the PDF file.
* Make sure all cells are executed so all your code and its results are included.
* Double-check that the PDF displays correctly before you submit it.

# Downloads and extracts the data. These commands download and extract the
# email datasets from a specified source.
# They work in a Linux environment, which is the base of Google Colab. ASDF

In [ ]:
# Downloading datasets

!wget https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
!wget https://spamassassin.apache.org/old/publiccorpus/20021010_spam.tar.bz2

# Extracting the datasets

!tar -xjf 20021010_easy_ham.tar.bz2
!tar -xjf 20021010_hard_ham.tar.bz2
!tar -xjf 20021010_spam.tar.bz2

--2023-11-27 16:10:33--  https://spamassassin.apache.org/old/publiccorpus/20021010_easy_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to spamassassin.apache.org (spamassassin.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1677144 (1.6M) [application/x-bzip2]
Saving to: ‘20021010_easy_ham.tar.bz2.3’

20021010_easy_ham.t 100%[===================>]   1.60M  --.-KB/s    in 0.01s   

2023-11-27 16:10:33 (144 MB/s) - ‘20021010_easy_ham.tar.bz2.3’ saved [1677144/1677144]

--2023-11-27 16:10:33--  https://spamassassin.apache.org/old/publiccorpus/20021010_hard_ham.tar.bz2
Resolving spamassassin.apache.org (spamassassin.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to spamassassin.apache.org (spamassassin.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1021126 (997K) [application/x-bzip2]
Saving to: ‘20021010_hard

The data is now in the following three folders: `easy_ham`, `hard_ham`, and `spam`. You can confirm this via the following command:

In [ ]:
# Listing the downloaded files

!ls -lah

total 16M
drwxr-xr-x 1 root root 4.0K Nov 27 16:10 .
drwxr-xr-x 1 root root 4.0K Nov 27 15:16 ..
-rw-r--r-- 1 root root 1.6M Jun 29  2004 20021010_easy_ham.tar.bz2
-rw-r--r-- 1 root root 1.6M Jun 29  2004 20021010_easy_ham.tar.bz2.1
-rw-r--r-- 1 root root 1.6M Jun 29  2004 20021010_easy_ham.tar.bz2.2
-rw-r--r-- 1 root root 1.6M Jun 29  2004 20021010_easy_ham.tar.bz2.3
-rw-r--r-- 1 root root 998K Dec 16  2004 20021010_hard_ham.tar.bz2
-rw-r--r-- 1 root root 998K Dec 16  2004 20021010_hard_ham.tar.bz2.1
-rw-r--r-- 1 root root 998K Dec 16  2004 20021010_hard_ham.tar.bz2.2
-rw-r--r-- 1 root root 998K Dec 16  2004 20021010_hard_ham.tar.bz2.3
-rw-r--r-- 1 root root 1.2M Jun 29  2004 20021010_spam.tar.bz2
-rw-r--r-- 1 root root 1.2M Jun 29  2004 20021010_spam.tar.bz2.1
-rw-r--r-- 1 root root 1.2M Jun 29  2004 20021010_spam.tar.bz2.2
-rw-r--r-- 1 root root 1.2M Jun 29  2004 20021010_spam.tar.bz2.3
drwxr-xr-x 4 root root 4.0K Nov 21 14:21 .config
drwx--x--x 2  500  500 168K Oct 10  2002 easy_ha

### 1. Preprocessing:

In the pre-processing section, the code outputs the number of emails in each of the datasets:
1. Easy ham
2. Spam
3. Hard ham

These print statements provide a clear understanding of the volume of data that the model will be processing.
By knowing the number of emails in each category, one can gauge the dataset's diversity and size, which are vital factors in determining the robustness and reliability of the training process.
This information ca be good to know to understand how it influences the model's learning and eventual performance.

Note: The email files contain a lot of extra information, besides the actual message. This will momentaily be ignored meaning that the code will be ran on the entire text.

We don’t want to train and test on the same data.An example that explains the reason for this is the following:

Training and testing on the same data is akin to memorizing answers for an upcoming test. You will only be good at answering these specific questions but not good at other questions in the same topic.

In other words, our approach gives us a more accurate measure of how well the model generalizes to new data, which is the ultimate goal of most machine learning models.

 1. Split the spam and ham datasets into a training set and a test set. (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`). Use `easy_ham` for quesions 1 and 2.


In [ ]:
# Used for accessing the file system.
import os

# For splitting data into train and test sets
from sklearn.model_selection import train_test_split

# Function to read files from a directory into a list

def read_email_files(directory):
  # Reads email files from a given directory and returns them in a list

    files = os.listdir(directory) # Lists all files in the directory
    emails = []
    for file in files:
        with open(os.path.join(directory, file), 'r', encoding='latin1') as email_file:
            emails.append(email_file.read()) # Reads each file and appends its content
    return emails

# Reading and storing emails from directories
easy_ham = read_email_files('/content/easy_ham')
spam = read_email_files('/content/spam')
hard_ham = read_email_files('/content/hard_ham')

# Splitting data into training and test sets
hamtrain, hamtest = train_test_split(easy_ham, test_size=0.2, random_state=42)
spamtrain, spamtest = train_test_split(spam, test_size=0.2, random_state=42)

# Prints amount of email files in respective folders
print('Easy ham:', + len(easy_ham))
print('Spam:', + len(spam))
print('Hard ham:', + len(hard_ham))


Easy ham: 2551
Spam: 501
Hard ham: 250


The code cell's output shows the amount of email files in the three respective folders.

### 2. Write a Python program:

As we move to the model evaluation outputs, the evaluate_model function comes into play, providing a detailed look at the model's performance through two key metrics: True Positives and False Negatives.

The Python program should be written in the steps below:

1.	Uses the four datasets from Question 1 (`hamtrain`, `spamtrain`, `hamtest`, and `spamtest`).
2.	Trains a Naïve Bayes classifier (use the [scikit-learn library](https://scikit-learn.org/stable/)) on `hamtrain` and `spamtrain`, that classifies the test sets and reports True Positive and False Negative rates on the `hamtest` and `spamtest` datasets. You can use `CountVectorizer` ([documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer)) to transform the email texts into vectors. Please note that there are different types of Naïve Bayes Classifiers available in *scikit-learn* ([Documentation here](https://scikit-learn.org/stable/modules/naive_bayes.html)). Here, you will test two of these classifiers that are well suited for this problem:
- Multinomial Naive Bayes
- Bernoulli Naive Bayes.



In [ ]:
# Needed modules to perform task 2:

from sklearn.feature_extraction.text import CountVectorizer # Converts text to numerical vector
from sklearn.naive_bayes import MultinomialNB, BernoulliNB # Naive Bayes models
from sklearn.metrics import confusion_matrix # For model evaluation


train_emails = hamtrain + spamtrain # Combining training datasets
test_emails = hamtest + spamtest # Combining test datasets


train_labels = [0]*len(hamtrain) + [1]*len(spamtrain) # Creating labels for training
test_labels = [0]*len(hamtest) + [1]*len(spamtest) # Creating labels for testing

# Convert emails to a matrix of token counts

vectorizer = CountVectorizer() # Initializing the vectorizer
X_train = vectorizer.fit_transform(train_emails) # Vectorizing training data
X_test = vectorizer.transform(test_emails) # Vectorizing test data

# Training Multinomial Naive Bayes model
mnb = MultinomialNB()
mnb.fit(X_train, train_labels)

# Training Bernoulli Naive Bayes model
bnb = BernoulliNB()
bnb.fit(X_train, train_labels)

# Making predictions with both models
mnb_pred = mnb.predict(X_test)
bnb_pred = bnb.predict(X_test)

# Evaluate the models

def evaluate_model(predictions, labels):
    # Evaluates the model by calculating True Positives and False Negatives
    tn, fp, fn, tp = confusion_matrix(labels, predictions).ravel()
    print("Amount of True Positives (TP):", tp)
    print("Amount of False Negatives (FN):", fn)
    print("True Positive Rate:", str(round(tp / (tp + fn), 2) * 100) + "%")
    print("False Negative Rate:", str(round(fn / (tp + fn), 2) * 100) + "%")

# Evaluating both models

print("Multinomial Naive Bayes:")
evaluate_model(mnb_pred, test_labels)
print("\nBernoulli Naive Bayes:")
evaluate_model(bnb_pred, test_labels)

Multinomial Naive Bayes:
Amount of True Positives (TP): 72
Amount of False Negatives (FN): 29
True Positive Rate: 71.0%
False Negative Rate: 28.999999999999996%

Bernoulli Naive Bayes:
Amount of True Positives (TP): 38
Amount of False Negatives (FN): 63
True Positive Rate: 38.0%
False Negative Rate: 62.0%


The number of True Positives represents the emails correctly identified as spam by the model, a direct measure of the model's accuracy and effectiveness in recognizing spam emails.

On the other hand, False Negatives denote the spam emails that the model failed to identify, slipping through as false 'ham' or non-spam emails. This metric is equally important as it highlights the model's shortcomings and areas where it might be improved.

These metrics demonstrates the model's performance, highlighting both its strengths in identifying spam and its weaknesses where improvements might be necessary.

### 3. Run on hard ham:
Run the two models from Question 2 on `spam` versus `hard-ham`, and compare to the `easy-ham` results.




In [ ]:
train_emails_hard_ham = hard_ham + spam # Combining hard ham and spam for training
train_labels_hard_ham = [0] * len(hard_ham) + [1] * len(spam)

# Vectorize the training data
vectorizer = CountVectorizer()
X_train_hard_ham = vectorizer.fit_transform(train_emails_hard_ham) # Vectorizing hard ham data

# Trains the Multinomial and Bernoulli Naive Bayes models on hard ham data
mnb_hard_ham = MultinomialNB().fit(X_train_hard_ham, train_labels_hard_ham)
bnb_hard_ham = BernoulliNB().fit(X_train_hard_ham, train_labels_hard_ham)

X_test = vectorizer.transform(hamtest + spamtest)  # Transforming test data

# Labels for the test set
test_labels = [0] * len(hamtest) + [1] * len(spamtest)

# Predicting using models trained on hard ham
mnb_pred_hard_ham = mnb_hard_ham.predict(X_test)
bnb_pred_hard_ham = bnb_hard_ham.predict(X_test)

# Function to evaluate the model

def evaluate_model(predictions, actual):
    tn, fp, fn, tp = confusion_matrix(actual, predictions).ravel()
    print("True Positives (TP):", tp)
    print("False Negatives (FN):", fn)
    print("True Positive Rate:", str(round(tp / (tp + fn), 2) * 100) + "%")
    print("False Negative Rate:", str(round(fn / (tp + fn), 2) * 100) + "%")

# Evaluating models trained on hard ham
print("Multinomial Naive Bayes:")
evaluate_model(mnb_pred_hard_ham, test_labels)
print("\nBernoulli Naive Bayes:")
evaluate_model(bnb_pred_hard_ham, test_labels)

Multinomial Naive Bayes:
True Positives (TP): 101
False Negatives (FN): 0
True Positive Rate: 100.0%
False Negative Rate: 0.0%

Bernoulli Naive Bayes:
True Positives (TP): 101
False Negatives (FN): 0
True Positive Rate: 100.0%
False Negative Rate: 0.0%


In this section, the models, which were initially trained on the hard ham datasets, are put to the test against the test set. This step is essential to gauge the models' adaptability and effectiveness in dealing with more challenging and nuanced scenarios. Hard ham emails are typically more difficult to distinguish from spam, thus presenting a more rigorous test for the model.

Evaluating the model's performance on this dataset provides insights into how well the model can adapt and perform when faced with complex, real-world data that doesn't always present clear-cut categorizations. This evaluation is important for  understanding the robustness and real-world applicability of the models in question.